<a href="https://colab.research.google.com/github/johyunkang/python-ml-guide/blob/main/python_ml_perfect_guide_08_TextAnal_07OpinionReview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 07 문서 군집화 소개와 실습 (Opinion Review 데이터 세트)

- 문서 군집화 개념 : 비슷한 텍스트 구성의 문서를 군집화 하는 것
- 텍스트 분류 기반의 문서 분류는 학습 데이터 세트가 필요하지만
- 문서 군집화는 학습 데이터 세트가 필요 없는 비지도학습 기반으로 동작


- 데엍 세트 URL : https://archive.ics.uci.edu/ml/datasets/Opinosis+Opinion+%26frasl%3B+Review

In [4]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

In [5]:
import pandas as pd
import glob, os

# path directory 설정
path = r'/content/drive/MyDrive/Colab Notebooks/data/text-anal/topics'

# path로 지정한 디렉토리 밑에 있는 모든 .data 파일을 리스트로 취합
all_files = glob.glob(os.path.join(path, "*.data"))
filename_list = []
opinion_text = []

# 개별 파일명은 filename_list 로 취합
# 개별 파일의 파일 내용은 DataFrame 로딩 후 다시 String 으로 변환해 opinion_text list 로 취합
for file_ in all_files :
    # 개별 파일을 읽어서 DF 로 생성
    df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')

    # 절대 경로로 주어진 파일명을 가공.
    # 마지막 확장자 .data 제거
    filename_ = file_.split('/')[-1]
    filename = filename_.split('.')[0]

    # 파일명 list와 파일내용 list에 파일명과 파일 내용을 추가
    filename_list.append(filename)
    opinion_text.append(df.to_string())

# 파일명 list와 파일 내용 list 객체를 DataFrame으로 생성
document_df = pd.DataFrame({'filename': filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,battery-life_ipod_nano_8gb,...
1,video_ipod_nano_8gb,...
2,directions_garmin_nuvi_255W_gps,...
3,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...
4,screen_ipod_nano_8gb,...


- TF-IDF 형태로 피처 벡터화
- 아래 함수 LemTokens(), LemNormalize() 는 아래 링크 참조
- https://github.com/wikibook/pymldg-rev/blob/master/8%EC%9E%A5/8.7%20%EB%AC%B8%EC%84%9C%20%EA%B5%B0%EC%A7%91%ED%99%94%20%EC%86%8C%EA%B0%9C%EC%99%80%20%EC%8B%A4%EC%8A%B5(Opinion%20Review%20%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EC%84%B8%ED%8A%B8).ipynb


In [6]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english', ngram_range=(1, 2), min_df=0.05, max_df=0.85)

# opinion_text 컬럼 값으로 피처 벡터화 수행
feature_vect = tfidf.fit_transform(document_df['opinion_text'])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  % sorted(inconsistent)


- 군집화 기법은 K-평균을 이용
- 문서의 유형은 크게 보면 전자제품, 자동차, 호텔로 되어 있음
- 먼저 5개의 중심(Centroid) 기반으로 어떻게 군집화되는지 확인해 보겠음

In [8]:
from sklearn.cluster import KMeans

# 5개의 집합으로 군집화 수행
kmcluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
kmcluster.fit(feature_vect)
cluster_label = kmcluster.labels_
cluster_centers = kmcluster.cluster_centers_

- 군집의 레이블을 파일명과 파일 내용을 가지고 있는 DF 에 cluster_label 컬럼을 추가 저장

In [9]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,battery-life_ipod_nano_8gb,...,2
1,video_ipod_nano_8gb,...,2
2,directions_garmin_nuvi_255W_gps,...,4
3,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,2
4,screen_ipod_nano_8gb,...,2


p. 520